In [ ]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=20)
client

In [ ]:
import time
import random

def inc(x):
    time.sleep(random.random())
    return x + 1

def double(x):
    time.sleep(random.random())
    return 2 * x
    
def add(x, y):
    time.sleep(random.random())
    return x + y 

In [ ]:
future = client.submit(inc, 1)  # returns immediately with pending future


In [ ]:
future
future.result()

In [ ]:
x = client.submit(inc, 1)
y = client.submit(double, 2)
z = client.submit(add, x, y)

In [ ]:
z.result()

In [ ]:
zs = []

In [ ]:
%%time

for i in range(1024):
    x = client.submit(inc, i)     # x = inc(i)
    y = client.submit(double, x)  # y = inc(x)
    z = client.submit(add, x, y)  # z = inc(y)
    zs.append(z)

In [ ]:
zs

In [ ]:
total = client.submit(sum, zs)

In [ ]:
L = zs
while len(L) > 1:
    new_L = []
    for i in range(0, len(L), 2):
        future = client.submit(add, L[i], L[i + 1])  # add neighbors
        new_L.append(future)
    L = new_L  

In [ ]:
del future, L, new_L, total  # clear out some old work

In [ ]:
from dask.distributed import as_completed

zs1 = client.map(inc, zs)
seq = as_completed(zs1)

while seq.count() > 1:  # at least two futures left
    a = next(seq)
    b = next(seq)
    new = client.submit(add, a, b, priority=1)  # add them together
    seq.add(new)                                # add new future back into loop